<span style='color:#0066cc'> <span style='font-family:serif'> <font size="13"> **Access OSCAR Data with OPeNDAP**<span style='color:#0066cc'>

<span style='color:#0066cc'><font size="5"> **About the "Ocean Surface Current Analyses Real-time (OSCAR)" Version 2 [OSCAR v2.0](https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_FINAL_V2.0) data**
1. <font size="3"><span style='color:Black'> The OSCAR project produces global near-surface current analyses computed from satellite
observations: ocean surface topography, ocean vector winds and sea surface temperature.
2. <font size="3"><span style='color:Black'> OSCAR ocean mixed layer velocities are calculated from satellite-sensed sea surface height gradients, ocean vector winds, and sea surface temperature gradients using a simplified physical model for geostrophy, Ekman, and thermal wind dynamics



<span style='color:#ff6666'><font size="5">**Requirements**
1. <font size="3"><span style='color:Black'> Have a Bearer Token for EarthData in the Cloud (See `GetStarted` Notebook).
2. <font size="3"><span style='color:Black'> Upload the Bearer Token from local file `token.json`
3. <font size="3"><span style='color:Black'> Data covers 1993-Jan-01 to 2022-Aug-05

**Source**: [OSCAR V2 Guide](https://deotb6e7tfubr.cloudfront.net/s3-edaf5da92e0ce48fb61175c28b67e95d/podaac-ops-cumulus-docs.s3.us-west-2.amazonaws.com/oscar/open/L4/oscar_v2.0/docs/oscarv2guide.pdf?A-userid=None&Expires=1748882398&Signature=YjGSgWfKq3IJpfsxqkNdyEOGGjQpGiWKPXaf3TGQgSAIbQM9mnO-g0jAvx4VHJoFHPHxI7nH0x-AtJLO6ipi3U3TT4CddFhQxn3ZMhj50UdEFsCkARm8hP2j32evdbddhSbu3TJFwZkaXLvW0FjMaRI3xufHzS~Gw8hP-ubPS0SFHgO8jwRMaDA71V85WMk0~wQyPh25pbDAcg24wPmw19vgAJ-jhRprGbKj5QcVHa~4cOwQg537lIDWPYZiPFWZBooVvJTWw1a0ohpjZa98kUsZ6Yo7TRThHDwnxnnT5eriazhGessgi-Ibz-RQSTFoDc~3gpRDWq~C1Q-SmP-ItA__&Key-Pair-Id=K299NXKZAIEHE5), and https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_FINAL_V2.0


    

In [ ]:
from pydap.net import create_session
from pydap.client import get_cmr_urls, consolidate_metadata, open_url
import xarray as xr
import datetime as dt
import numpy as np
import json
import matplotlib.pyplot as plt

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 


<font size="3.5"> Here we use the Bearer Token to create an authenticated session. The Bearer token should be stored on a local json file, after completed the `GetStarted` Notebook.



In [ ]:
# load token json data
with open('token.json', 'r') as fp:
    token = json.load(fp)

# pass Token Authorization to a new Session.
my_session = create_session(use_cache=True, session_kwargs=token)
my_session.cache.clear()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Query opendap urls using NASA's CMR API**

In [ ]:
oscar_ccid = "C2098858642-POCLOUD"

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Filter data via Temporal Searches**

<font size="3.5"> Users can specify date ranges  NASA's CMR can 

<font size="3.5"> There are two ways to specify formats.

    1. Using `python`'s datetime package. It follows the `year-month-day` formatting
    2. A string with the following format: YYYY-MM-DDTHH:MM:SSZ


In [ ]:
time_range=[dt.datetime(2020, 1, 1), dt.datetime(2020, 1, 31)] # One month of data

In [ ]:
url_limits = 100 # controls the max number of urls returns. Default is 50

In [ ]:
urls = get_cmr_urls(ccid=oscar_ccid,time_range=time_range, limit=url_limits) # you can incread the limit of results
len(urls)

In [ ]:
dap4_urls = [url.replace("https", "dap4") for url in urls]

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Consolidate metadata**

<font size="3.5"> All URLs belonging to the same Collection share many identical variables and metadata. The following function
reduces redundant metadata


OSCAR data - dimensions and coordinates (e.g. `lat` and `latitude`) do not match in name exactly. As a result, to speed up the process of dataset generation, we will need to specify an extra key parameter `set_maps=True` below.


In [ ]:
%%time
consolidate_metadata(dap4_urls, concat_dim='time', safe_mode=False, set_maps=True, session=my_session)

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Create Virtual Aggregated Dataset with Xarray**

<font size="3.5"> Now, you can create a virtually aggregated view of the dataset that is ready to analyze with Xarray and Pydap as an engine.


In [ ]:
%%time
ds = xr.open_mfdataset(dap4_urls, engine='pydap', session=my_session, combine='nested', concat_dim="time")
ds

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Sample by coordinates**

<font size="3.5"> The dimensions `latitude`, `longitude` do not match the name of the coordinates, despite both being 1D arrays. However, to use these we can load the coordinates `lat` and `lonat  into memory and query the indexes longitude and latitude that match our coordinates of interest.

<font size="3.5"> We will need:
- <font size="3.5"> Load `lat`, `lon` into memory, using the xarray `.load()` method
- <font size="3.5"> Check the valid range for each `lat` and `lon` coordinate. In the case of OSCAR data, `lon>=0`. If our in situ data has negative values, we will need to transform lon values to match the OSCAR value ranges.
- <font size="3.5"> Identify the indexes of `longitude` and `longitude` that match our coordinate values
- <font size="3.5"> Sample the data using these indexes.

In [ ]:
# start with these fake in situ coordinate data of interest
insitu_lat = np.array([0, 10, -20, 70, -50, 32, 65])
insitu_lon = np.array([0, 120, 200, 300, 100, 100, 260])

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **load coordinates into memory**


In [ ]:
lon, lat = ds['lon'].load(), ds['lat'].load()

In [ ]:
ds

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Find closes value match**


In [ ]:
def find_closest_indices(array, target_values):
    """
    Finds the indices of the closest matching values in a NumPy array for multiple target values.

    Args:
        array (np.ndarray): The NumPy array to search in.
        target_values (list or np.ndarray): A list or array of target values.

    Returns:
        np.ndarray: A NumPy array containing the indices of the closest matching values.
    """
    indices = []
    for target_value in target_values:
        abs_diff = np.abs(array - target_value)
        closest_index = np.argmin(abs_diff)
        indices.append(closest_index)
    return np.array(indices)

In [ ]:
lon.values

### check these produce sensible values

In [ ]:
Ilon = find_closest_indices(lon.values, insitu_lon)
Ilat = find_closest_indices(lat.values, insitu_lat)

In [ ]:
ds['lat'].isel(latitude=Ilat)

In [ ]:
ds['lon'].isel(longitude=Ilon) 

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Sample variables of interest**

<font size="3.5"> Will create a list of all `U` and `V` velocities that span the entire time range of interesst, and that match a specific `latitude`, `longitude` pair. 


In [ ]:
%%time
U, V = [], []
for i in range(len(Ilat)):
    U += [ds['u'].isel(latitude=Ilat[i], longitude=Ilon[i]).load()]
    V += [ds['v'].isel(latitude=Ilat[i], longitude=Ilon[i]).load()]


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Inspect sampled data**

<font size="3.5"> Each element of the `U`, `V` list yields a `u` and `v` velocity DataArray that spans all times of interest (time-dependent). Each element of the `u` contains information about the coordinates it was sampled from!

<font size="3.5"> For example: check the first element


In [ ]:
u, v = U[0], V[0]

In [ ]:
u

In [ ]:
for i in range(len(U)):
    print('longitude: ', U[i].coords['lon'].values, " ,latitude:", U[i].coords['lat'].values)
          

In [ ]:
u.time.values